## This Notebook shows how a raw Dataset gets processed for further machine learning steps

In [14]:
# Imports
import os, sys
import numpy as np
from typing import Tuple, List, Dict
import pandas as pd
import io

# to enable local imports
module_path = os.path.abspath('code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from machine_learning_data_generation import loadPickeldDataset
from consts import TARGET_LABEL_DICT, DEVICES_NEUROSCAN

D:\Masterthesis\thesis_eeg\code


## Save a dataset to pickel files
With this function  a dataset gets processed and the output will be saved to pickel files

In [4]:
PROCESS_DATA = False

if PROCESS_DATA:
    # Process the online EEG Data
    processRawDatasetToPickleFiles(datasetDirPath = "D:/Masterthesis/EEG_Data/eeg_data_online",
                              device = DEVICES_NEUROSCAN,
                              awakeFileName = None,
                              fatigueFileName = "Fatigue_state_256hz.csv",
                              normalFileName = "Normal_state_256hz.csv",
                              unlabeledFileName = None)
else:
    print ("Already processed the EEG Online Data")

Already processed the EEG Online Data


### Function to create a Machine Learning Dataset
With this functions you can create a X and y Dataset from a given EEG Dataset (use the function 'processRawDatasetToPickleFiles')

**Todo - Create X und Y for feature DF**

In [5]:
CREATE_ML_DATA = False

if CREATE_ML_DATA:
    
    eegDataset = loadPickeldDataset("D:/Masterthesis/EEG_Data/eeg_data_online")
    
    createAndSafeMlDataset(eegDataset=eegDataset,
                           targetLabelDict=TARGET_LABEL_DICT,
                           dirPath="D:/Masterthesis/EEG_Data/eeg_data_online")
else:
    print("Already created ML Data")


Already created ML Data


### Example - Load  the online EEG Dataset

In [27]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

def convertFeatureDfToXy(featureDf : pd.DataFrame, target : int) -> (np.ndarray, np.ndarray):
    samples = []
    targetArray = []
    
    for index, row in featureDf.iterrows():
        timesteps = []
        features = row.to_numpy() # features
        timesteps.append(features)
        samples.append(timesteps)
        
        targetArray.append(target)
    
    X = np.array(samples)
    y = np.array(targetArray)
    
    return X, y 
    
def loadOnlineEEGdata(test_size=0.3, shuffle=False):
    dirPath = "D:/Masterthesis/EEG_Data/eeg_data_online"
    
    print("Loading Online EEG Data from {} ...".format(dirPath))
    
    # load array
    X = np.load(os.path.join(dirPath, 'data_series_X.npy'))
    y = np.load(os.path.join(dirPath, 'data_series_y.npy'))
    
    # load target labels
    
    # load feature names
    
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=109, shuffle=shuffle) # 70% training and 30% test
    
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    return X_train, y_train, X_test, y_test

In [28]:
X_train, y_train, X_test, y_test = loadOnlineEEGdata()

Loading Online EEG Data from D:/Masterthesis/EEG_Data/eeg_data_online...
(5024, 512, 40) (5024, 2) (2154, 512, 40) (2154, 2)


In [61]:
np.dstack(np.dstack(X_train))[0].shape

(5024, 512)

In [60]:
np.dstack(np.dstack(X_test)).shape

(40, 2154, 512)

In [49]:
X_train_test[2].shape

(5024, 512)

In [50]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [52]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [53]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [121]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
        
    print("n_timesteps: {} - n_features: {} - n_outputs: {}".format(n_timesteps, n_features, n_outputs))
    model = Sequential()
    
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=10, channel=None):
    # load data
    trainX, trainy, testX, testy = loadOnlineEEGdata()
    
    if channel != None:
        print("Using only Channel: {}".format(channel))
        trainX = np.dstack(np.dstack(trainX))[channel]
        testX = np.dstack(np.dstack(testX))[channel]
    
    print("Loaded the Dataset")
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)


In [64]:
#run_experiment(repeats=1, channel=5)

In [116]:
trainX, trainy, testX, testy = loadOnlineEEGdata()

Loading Online EEG Data from D:/Masterthesis/EEG_Data/eeg_data_online...
(5024, 512, 40) (5024, 2) (2154, 512, 40) (2154, 2)


In [117]:
channel = 5
#trainX = np.dstack(np.dstack(trainX))[channel]
#testX = np.dstack(np.dstack(testX))[channel]
trainX = trainX[:, :, channel:channel+1]
testX = testX[:, :, channel:channel+1]


(5024, 512, 1)

In [120]:
print("Shape trainX: {}".format(trainX.shape))
print("Shape testX: {}".format(testX.shape))

repeats = 1
# repeat experiment
scores = list()
for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
# summarize results
summarize_results(scores)

Shape trainX: (5024, 512, 1)
Shape testX: (2154, 512, 1)
n_timesteps: 512 - n_features: 1 - n_outputs: 2


12-May-20 11:43:31 | WARNING | From C:\Users\reiss\Anaconda3\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



>#1: 44.475
[44.47539448738098]


NameError: name 'mean' is not defined